# Book Recommender System

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# For Pre-Processing
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer

# For GNN
import torch
import torch.nn as nn
import torch.nn.functional as F

import dgl
import dgl.function as fn
from dgl.nn import GraphConv

# Data Exploration

In [3]:
ratings = pd.read_csv("Data/Ratings.csv")
print(f"Ratings Data Shape: {ratings.shape} \n")
print(f"{ratings.dtypes}\n")
print(ratings.head())

Ratings Data Shape: (1149780, 3) 

User-ID         int64
ISBN           object
Book-Rating     int64
dtype: object

   User-ID        ISBN  Book-Rating
0   276725  034545104X            0
1   276726  0155061224            5
2   276727  0446520802            0
3   276729  052165615X            3
4   276729  0521795028            6


In [4]:
books = pd.read_csv("Data/Books.csv")
print(f"Books Data Shape: {books.shape} \n")
print(f"{books.dtypes}\n")
# print(books.head(3))

Books Data Shape: (271360, 8) 

ISBN                   object
Book-Title             object
Book-Author            object
Year-Of-Publication    object
Publisher              object
Image-URL-S            object
Image-URL-M            object
Image-URL-L            object
dtype: object



/var/folders/0f/xqjk7pr56mjd12xvy1w435_40000gn/T/ipykernel_87175/2638443579.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("Data/Books.csv")


In [5]:
print('Nan-values by column')
print(books.isna().sum())

Nan-values by column
ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64


In [6]:
users = pd.read_csv("Data/Users.csv")
print(f"Users Data Shape: {users.shape} \n")

print(users.head())
print()
print(f"{users.dtypes}\n")

print('Nan-values by column')
print(users.isna().sum())

Users Data Shape: (278858, 3) 

   User-ID                            Location   Age
0        1                  nyc, new york, usa   NaN
1        2           stockton, california, usa  18.0
2        3     moscow, yukon territory, russia   NaN
3        4           porto, v.n.gaia, portugal  17.0
4        5  farnborough, hants, united kingdom   NaN

User-ID       int64
Location     object
Age         float64
dtype: object

Nan-values by column
User-ID          0
Location         0
Age         110762
dtype: int64


# Data Preprocessing

- We will only use users and books present in the ratings dataset 

### Renumber IDs to reduce inactive users

In [ ]:
# renumber IDs to reduce inactive users
lessen_user_ids = {userid: idx for idx, userid in enumerate(ratings['User-ID'].unique())}
ratings['New-User-ID'] = ratings['User-ID'].map(lessen_user_ids)
user_ids = list(ratings['New-User-ID'].unique())
num_users = len(set(user_ids))
print(f"Number of User IDs: {num_users}")

### Map book identifiers (ISBN) to a unique integer identifier for datatype compatibility of dgl

In [ ]:
# Map book identifiers (ISBN) to a unique integer identifier for datatype compatibility of dgl
isbn_to_id = {isbn: idx for idx, isbn in enumerate(ratings['ISBN'].unique())}
ratings['Book-ID'] = ratings['ISBN'].map(isbn_to_id)
book_ids = list(ratings['Book-ID'].unique())
num_books = len(set(book_ids))

print(f'There are {len(user_ids)} unique users, and {len(book_ids)} unique books in the ratings dataset.')

### Remove users and books not included in the ratings dataset

In [ ]:
# Remove users and books not included in the ratings dataset
books['Book-ID'] = books['ISBN'].map(isbn_to_id)
books_clean = books[books['Book-ID'].isin(book_ids)]
books_clean_ids = books_clean['Book-ID'].unique()
percent_books_missing = (num_books-len(books_clean_ids))/num_books*100

print(f'There are {round(percent_books_missing, 2)}% of books in the graph missing in the books data')

#### Retaining users who have at least one book

In [ ]:
users['New-User-ID'] = users['User-ID'].map(lessen_user_ids)
users_clean = users[users['New-User-ID'].isin(user_ids)]

#### Fill in missing value Age with simple imputer

In [ ]:
knn_imputer = KNNImputer(n_neighbors=5)
users_clean['Age'] = knn_imputer.fit_transform(users_clean[['Age']])

1/5 of the books that have rating information do not have further information on the books dataset. However, as our objective is to investigate an user-based recommender system, this is irrelevant. We are able to embed the age and location data of users. As the age data is sparse, location data will be our main source of information.

In [ ]:
# Make graph
src = torch.tensor(ratings['New-User-ID'].values)
dst = torch.tensor(ratings['Book-ID'].values)
print(len(src), len(dst))

edges = {
    ('user', 'rating', 'book'): (src, dst)
}

g = dgl.heterograph(edges, num_nodes_dict={'user': num_users, 'book': num_books})
print(g)

In [ ]:
# Weigth the edges by ratings
rating_data = ratings['Book-Rating'].values
ratings_tensor = torch.tensor(rating_data, dtype=torch.float32)
g.edges['rating'].data['rating'] = ratings_tensor 

In [ ]:
# # Add age to user feature
# ages = users_clean['Age'].values
# ages_tensor = torch.tensor(ages, dtype=torch.float32)
# g.nodes['user'].data['age'] = ages_tensor

# Add age to user feature
ages = users_clean.set_index('New-User-ID')['Age'].sort_index().values
ages_tensor = torch.tensor(ages, dtype=torch.float32)
g.nodes['user'].data['age'] = ages_tensor

We extract the country from the location by obtaining the expression after the last comma in e.g. nyc, new york, usa

In [ ]:
users_clean['Country'] = users_clean['Location'].str.rsplit(',', n=1).str[-1].str.strip()
country_counts = users_clean['Country'].value_counts(normalize=True)

# We see that less frequent locations do not always contain country names, so we remove values of locations representing less than 1%
rare_countries = country_counts[country_counts < 0.01].index
users_clean.loc[users_clean['Country'].isin(rare_countries), 'Country'] = np.nan

# Encoding Countries to a unique interger (same as label encoding)
country_ids = {country: idx for idx, country in enumerate(users_clean['Country'].unique())}  # map country to a unique integer
users_clean['CountryId'] = users_clean['Country'].map(country_ids)

country_ids = {country: idx for idx, country in enumerate(users_clean['Country'].dropna().unique())}
users_clean['CountryId'] = users_clean['Country'].map(country_ids).fillna(-1).astype(int)

countries = users_clean['CountryId'].values

# countries = users_clean.set_index('New-User-ID')['CountryId'].sort_index().fillna(-1).values  # Fill NaN with -1

countries_tensor = torch.tensor(countries, dtype=torch.float32)
g.nodes['user'].data['country'] = countries_tensor

In [ ]:
print(g.nodes['user'])
print(g.nodes['book'])

### Basic graph info

In [ ]:
print(g)  # Prints the basic info of the graph, such as number of nodes and edges per type

# Print number of nodes for each type
print("Number of users:", g.number_of_nodes('user'))
print("Number of books:", g.number_of_nodes('book'))

# Print number of edges
print("Number of ratings:", g.number_of_edges('rating'))

### Node and Edge feature inspection

In [ ]:
# Print user node features
print("User features:", g.nodes['user'].data.keys())

# Print book node features, if any
print("Book features:", g.nodes['book'].data.keys())

# Print edge features
print("Edge features:", g.edges['rating'].data.keys())

# Example to print specific feature details:
print("Sample user ages:", g.nodes['user'].data['age'][:5])  # prints first 5 user ages
print("Sample ratings:", g.edges['rating'].data['rating'][:5])  # prints first 5 ratings

### Eliminate isolated nodes if any

In [ ]:
compact_g = dgl.compact_graphs(g)

### Create synthetic features for book based on degree of the node

In [ ]:
book_in_degrees = compact_g.in_degrees(etype=('user', 'rating', 'book')).float().unsqueeze(1)

In [ ]:
compact_g.nodes['book'].data['in_degree'] = book_in_degrees

# Creating the GNN

## Architecture

In [ ]:
# class GNNRecommender(nn.Module):
#     def __init__(self, user_feats, book_feats, hidden_size, num_classes):
#         super(GNNRecommender, self).__init__()
#         self.user_conv = GraphConv(user_feats, hidden_size, allow_zero_in_degree=True)
#         self.book_conv = GraphConv(book_feats, hidden_size, allow_zero_in_degree=True)
#         self.fc = nn.Linear(hidden_size, num_classes)
        
#     def forward(self, g, user_features, book_features):
#         user_h = F.relu(self.user_conv(g, user_features))
#         book_h = F.relu(self.book_conv(g, book_features))
#         user_book_h = user_h + book_h
#         return self.fc(user_book_h)

class GNNRecommender(nn.Module):
    def __init__(self, user_feats_dim, book_feats_dim, hidden_size, num_classes):
        super(GNNRecommender, self).__init__()
        self.user_conv = GraphConv(user_feats_dim, hidden_size, allow_zero_in_degree=True)
        self.book_conv = GraphConv(book_feats_dim, hidden_size, allow_zero_in_degree=True)
        self.fc = nn.Linear(hidden_size * 2, num_classes)  # *2 for concatenation of user and book hidden states
        
        print(self.user_conv)
        print(self.book_conv)
        print(self.fc)
        
    def forward(self, g, user_features, book_features):
        user_h = F.relu(self.user_conv(g, user_features))
        book_h = F.relu(self.book_conv(g, book_features))
        
        # # Combine user and book features; here we concatenate
        # combined_h = torch.cat([user_h, book_h], dim=1)
        
        # Combine user and book features (this is just one way; ensure it aligns with your task)
        combined_h = torch.cat([user_h, book_h], dim=0)
        return self.fc(combined_h)

## Setting up model

In [ ]:
# CHANGED g. to compact_g.

# Create USER Features Tensor
age_tensor = compact_g.nodes['user'].data['age'].unsqueeze(1)
country_tensor = compact_g.nodes['user'].data['country'].unsqueeze(1)
user_feats = torch.cat([age_tensor, country_tensor], dim=1)

# Create BOOK Features Tensor
book_feats = compact_g.nodes['book'].data['in_degree']

user_feat_dim = user_feats.shape[1]  # the size of user feature
book_feat_dim = book_feats.shape[1]  # the size of book feature

print(f"User feature dimension {user_feat_dim}")
print(f"Book feature dimension {book_feat_dim}")

In [ ]:
hidden_size = 32
num_classes = 1  # predicting a single rating value
model = GNNRecommender(user_feat_dim, book_feat_dim, hidden_size, num_classes)

## Split graph for training and validation set

In [ ]:
def split_graph(g, proportion=0.8):
    # Split edges randomly for training and validation
    num_edges = g.number_of_edges('rating')
    all_edges = np.arange(num_edges)
    np.random.shuffle(all_edges)
    
    train_size = int(num_edges * proportion)
    train_edges = all_edges[:train_size]
    val_edges = all_edges[train_size:]
    
    # # Create subgraphs based on the edges (change to True)
    g_train = dgl.edge_subgraph(g, train_edges, relabel_nodes=True)
    g_val = dgl.edge_subgraph(g, val_edges, relabel_nodes=True)
    
    # Create subgraphs based on the edges
    # g_train = dgl.edge_subgraph(g, {'rating': train_edges}, relabel_nodes=False)
    # g_val = dgl.edge_subgraph(g, {'rating': val_edges}, relabel_nodes=False)
    
    return g_train, g_val

In [ ]:
g_train, g_val = split_graph(compact_g, proportion=0.8)

# Verify subgraphs
print("Number of users in training graph:", g_train.number_of_nodes('user'))
print("Number of books in training graph:", g_train.number_of_nodes('book'))
print("Number of ratings in training graph:", g_train.number_of_edges('rating'), "\n")

print("Number of users in validation graph:", g_val.number_of_nodes('user'))
print("Number of books in validation graph:", g_val.number_of_nodes('book'))
print("Number of ratings in validation graph:", g_val.number_of_edges('rating'))

In [ ]:
# Inspect the structure of the edge data
print(g_train.edges['rating'].data)
print(g_val.edges['rating'].data)

In [ ]:
# Get the features and ratings for the Training Set
age_tensor_train = g_train.nodes['user'].data['age'].unsqueeze(1) # (N, 1)
country_tensor_train = g_train.nodes['user'].data['country'].unsqueeze(1) # (N, 1)

assert age_tensor_train.shape[0] == country_tensor_train.shape[0], "Mismatch in number of users"


# Training Set
user_features_train = torch.cat([age_tensor_train, country_tensor_train], dim=1) # (N, 2)
book_features_train = g_train.nodes['book'].data['in_degree'] # (M, 1)

ratings_train = g_train.edges['rating'].data['rating']

# Add user and book features
g_train.nodes['user'].data['features'] = user_features_train
g_train.nodes['book'].data['features'] = book_features_train


# Get the features and ratings for the Validation Set
age_tensor_val = g_val.nodes['user'].data['age'].unsqueeze(1) # (N_val, 1)
country_tensor_val = g_val.nodes['user'].data['country'].unsqueeze(1) # (N_val, 1)

assert age_tensor_val.shape[0] == country_tensor_val.shape[0], "Mismatch in number of validation users"

# Validation Set
user_features_val = torch.cat([age_tensor_val, country_tensor_val], dim=1) # (N_val, 2)
book_features_val = g_val.nodes['book'].data['in_degree'] # (M_val, 1)
ratings_val = g_val.edges['rating'].data['rating']

# Verify feature dimensions
print("User features train shape:", user_features_train.shape)
print("Book features train shape:", book_features_train.shape)
print("Ratings train shape:", ratings_train.shape, "\n")

print("User features val shape:", user_features_val.shape)
print("Book features val shape:", book_features_val.shape)
print("Ratings val shape:", ratings_val.shape)


In [ ]:
g_train.edges['rating'].data.keys()

## Train

In [ ]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
def train(model, g, user_features, book_features, labels, optimizer, criterion):
    model.train()
    optimizer.zero_grad()
    outputs = model(g, user_features, book_features)
    # Added .squeeze()
    loss = criterion(outputs.squeeze(), labels)
    loss.backward()
    optimizer.step()
    return loss.item()

In [ ]:
g_train

In [ ]:
num_epochs = 2  # or however many epochs you deem necessary

for epoch in range(num_epochs):
    loss = train(model, g_train, user_features_train, book_features_train, ratings_train, optimizer, criterion)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss}')

## Evaluate model

In [ ]:
def evaluate(model, g, user_features, book_features, labels, criterion):
    model.eval()
    with torch.no_grad():
        outputs = model(g, user_features, book_features)
        # Added .squeeze()
        loss = criterion(outputs.squeeze(), labels)
    return loss.item()

In [ ]:
# Validation loss
validation_loss = evaluate(model, g_val, user_features_val, book_features_val, ratings_val, criterion)
print(f'Validation Loss: {validation_loss}')